In [2]:
## Imports
!pip install tensorflow
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import scipy
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

  Using cached tensorflow-2.4.1-cp38-cp38-win_amd64.whl (370.7 MB)
Processing c:\users\zashr\appdata\local\pip\cache\wheels\a0\16\9c\5473df82468f958445479c59e784896fa24f4a5fc024b0f501\termcolor-1.1.0-py3-none-any.whl
  Using cached grpcio-1.32.0-cp38-cp38-win_amd64.whl (2.6 MB)
  Using cached absl_py-0.12.0-py3-none-any.whl (129 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached protobuf-3.15.7-py2.py3-none-any.whl (173 kB)
  Using cached tensorboard-2.4.1-py3-none-any.whl (10.6 MB)
  Using cached gast-0.3.3-py2.py3-none-any.whl (9.7 kB)
  Using cached tensorflow_estimator-2.4.0-py2.py3-none-any.whl (462 kB)
  Using cached flatbuffers-1.12-py2.py3-none-any.whl (15 kB)
Processing c:\users\zashr\appdata\local\pip\cache\wheels\5f\fd\9e\b6cf5890494cb8ef0b5eaff72e5d55a70fb5631600

In [3]:
## Load training data
train_images = np.load("train_x.npy")
train_labels = np.genfromtxt("train_y.csv", delimiter=",", names=True, dtype=int)["Label"]

KeyboardInterrupt: 

In [ ]:
## Load test data
test_images = np.load("test_x.npy")

In [ ]:
## Reshape data into single-pixel arrays
train_images = train_images.reshape(40000, 128, 128, 1)
test_images = test_images.reshape(10000, 128, 128, 1)

In [ ]:
## Initialize data generator and free up the massive npys
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True
    # vertical_flip=True,
    # validation_split=0.2
)
train_generator=train_datagen.flow(train_images, train_labels)
del train_images
del train_labels

In [ ]:
## Do the same for the test set
test_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True
    # vertical_flip=True,
    # validation_split=0.2
)
test_generator=test_datagen.flow(test_images)
# del test_images

In [ ]:
## Show first batch of images with their label

def show_train_image(image_index):
    show_image(train_images[image_index])
    print("Label: ", round(train_labels[image_index]))

# def show_image(arr):
#     two_d = (np.reshape(arr, (128, 128)) * 255).astype(np.uint8)
#     plt.imshow(two_d, interpolation='nearest')
#     plt.show()

# def show_from_gen(arr):
#     plt.imshow((arr * 255).astype(np.uint8), interpolation='nearest')
#     plt.show()

# for i in range(1):
#     arr = next(train_generator)
#     for j in range(6):
#         show_from_gen(arr[0][j])
#         print('Label: ', arr[1][j])

In [ ]:
## Build the model
seq = models.Sequential([
    layers.Conv2D(32, (3,3), activation="relu", strides=(2,2), input_shape=(128, 128, 1)),
    layers.MaxPooling2D((2,2)),
    # layers.Dropout(0.2),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    # # layers.Dropout(0.2),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    # layers.Conv2D(128, (3, 3), activation='relu'),
    # layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    # layers.Dropout(0.2),
    # layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(9e-4)),
    layers.Dropout(0.5),
    # layers.Dense(128, activation='relu'),
    # layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    # # layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])
seq.summary()

In [ ]:
# seq = models.load_model("seq4.keras")

## Compile and fit the mode
# l to the training set
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam

filename = "seq6.keras"

# seq.compile(optimizer=Adam(),
#            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#            metrics=['accuracy'])


checkpointing = ModelCheckpoint(filename, monitor='accuracy',
                                       verbose=1, save_best_only=True, mode='max')

# seq.fit(train_generator,
#                         epochs=20,
#                         workers=4,
#                         callbacks=[checkpointing],
#                         shuffle=True,
#                     )

In [ ]:
## Additional training
seq = models.load_model('seq5.keras')
seq.pop(); seq.pop(); seq.pop()
seq.add(layers.Dense(64, activation='relu', name="added_1"))
seq.add(layers.Dropout(0.5))
seq.add(layers.Dense(64, activation='relu', name="added_2"))
seq.add(layers.Dense(10, activation='softmax', name="added_3"))
seq.fit(train_generator,
                        epochs=20,
                        workers=4,
                        callbacks=[checkpointing],
                        shuffle=True,
                    )

In [ ]:
## Generating predictions on the test set

pred = models.load_model('seq5.keras')
pred.summary()
predictions = pred.predict(test_generator, verbose=1)

In [ ]:
print(predictions)
for i in range(10):
    show_image(test_images[i])
    print("Predicted label: ", np.argmax(predictions[i]))

In [1]:
id = np.arange(10000)
category = np.array([np.argmax(predictions[i]) for i in range(len(predictions))])
output = np.column_stack((id, category))
print(output)
np.savetxt(fname='predictions.csv', X=output, header='Id,Category', fmt="%d,%d", comments='')

NameError: name 'np' is not defined